### Manipulation of PEAKS de novo results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/202_TROCAS7_Fusion_Apr2021_DENOVO_149/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/202_TROCAS7_Fusion_Apr2021_DENOVO_149


In [4]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks202 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/202_TROCAS7_Fusion_Apr2021_DENOVO_149/202_TROCAS7_Fusion_Apr2021_DENOVO_149_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks202))

print(peaks202.columns)

# These columns mess things up- get rid of them

del peaks202['Fraction']
del peaks202['Scan']
del peaks202['Source File']
del peaks202['Tag Length']
del peaks202['PTM']
del peaks202['tag (>=0%)']
del peaks202['mode']
del peaks202['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks202.columns = columns

mean_len = peaks202['length'].mean()
print(mean_len)

# look at the dataframe
peaks202.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 244
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
10.307377049180328


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,VLEGNEQFLNAAK,98,13,716.8758,2,70.18,375000.0,1431.7358,0.8
1,LSSPATLNSR,98,10,523.2864,2,45.68,91500000.0,1044.5564,1.8
2,LSSPATLNSR,98,10,523.2860,2,46.19,91500000.0,1044.5564,1.0
3,LSSPATLDSR,98,10,523.7775,2,48.99,737000.0,1045.5403,0.1
4,LSSPATLNSR,97,10,523.2865,2,45.18,91500000.0,1044.5564,2.0


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks202['A'] = peaks202['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks202['C'] = peaks202['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks202['D'] = peaks202['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks202['E'] = peaks202['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks202['F'] = peaks202['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks202['G'] = peaks202['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks202['H'] = peaks202['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks202 output, there will be no isoleucines (they're lumped in with leucines)
peaks202['I'] = peaks202['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks202['K'] = peaks202['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks202['L'] = peaks202['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks202['M'] = peaks202['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks202['N'] = peaks202['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks202['P'] = peaks202['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks202['Q'] = peaks202['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks202['R'] = peaks202['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks202['S'] = peaks202['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks202['T'] = peaks202['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks202['V'] = peaks202['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks202['W'] = peaks202['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks202['Y'] = peaks202['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks202['c-carb'] = peaks202['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks202['m-oxid'] = peaks202['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks202['n-deam'] = peaks202['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks202['q-deam'] = peaks202['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks202['stripped_peptide'] = peaks202['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks202['stripped_length'] = peaks202['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks202['ptm-total'] = peaks202['c-carb'] + peaks202['m-oxid'] + peaks202['n-deam'] + peaks202['q-deam']

# calculate NAAF numerator for each peptide k
peaks202['NAAF_num.'] = peaks202['Area'] / peaks202['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks202.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/202_BY_T00_GD_DN/202_BY_T00_GD_DN50.csv")

# check out the results
peaks202.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,VLEGNEQFLNAAK,98,13,716.8758,2,70.18,375000.0,1431.7358,0.8,2,...,0,0,0,0,0,0,VLEGNEQFLNAAK,13,0,2.884615e+04
1,LSSPATLNSR,98,10,523.2864,2,45.68,91500000.0,1044.5564,1.8,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,9.150000e+06
2,LSSPATLNSR,98,10,523.2860,2,46.19,91500000.0,1044.5564,1.0,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,9.150000e+06
3,LSSPATLDSR,98,10,523.7775,2,48.99,737000.0,1045.5403,0.1,1,...,0,0,0,0,0,0,LSSPATLDSR,10,0,7.370000e+04
4,LSSPATLNSR,97,10,523.2865,2,45.18,91500000.0,1044.5564,2.0,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,9.150000e+06


In [6]:
# keep only stripped peptide I/L and NAAF
dn_202 = peaks202[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_202.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_202.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/202_BY_T00_GD_DN50_ILnaafs.csv")

dn_202.head()

,stripped_peptide,Area,NAAF_num.
0,VLEGNEQFLNAAK,375000.0,2.884615e+04
1,LSSPATLNSR,91500000.0,9.150000e+06
2,LSSPATLNSR,91500000.0,9.150000e+06
3,LSSPATLDSR,737000.0,7.370000e+04
4,LSSPATLNSR,91500000.0,9.150000e+06


In [7]:
# made a new dataframe that contains the suMN of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks202['A'].sum(),
        'C': peaks202['C'].sum(),
        'D': peaks202['D'].sum(),
        'E': peaks202['E'].sum(),
        'F': peaks202['F'].sum(),
        'G': peaks202['G'].sum(),
        'H': peaks202['H'].sum(),
        'I': peaks202['I'].sum(),
        'K': peaks202['K'].sum(),
        'L': peaks202['L'].sum(),
        'M': peaks202['M'].sum(),
        'N': peaks202['N'].sum(),
        'P': peaks202['P'].sum(),
        'Q': peaks202['Q'].sum(),
        'R': peaks202['R'].sum(),
        'S': peaks202['S'].sum(),
        'T': peaks202['T'].sum(),
        'V': peaks202['V'].sum(),
        'W': peaks202['W'].sum(),
        'Y': peaks202['Y'].sum(),
        'c-carb': peaks202['c-carb'].sum(),
        'm-oxid': peaks202['m-oxid'].sum(),
        'n-deam': peaks202['n-deam'].sum(),
        'q-deam': peaks202['q-deam'].sum(),
        'Total area': peaks202['Area'].sum(),
        'Total length': peaks202['stripped_length'].sum()
       }

totalpeaks202 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks202['% C w/ carb'] = totalpeaks202['c-carb'] / totalpeaks202['C'] 

# calculate percentage of M's that are oxidized
totalpeaks202['% M w/ oxid'] = totalpeaks202['m-oxid'] / totalpeaks202['M'] 

# calculate percentage of N's that are deamidated
totalpeaks202['% N w/ deam'] = totalpeaks202['n-deam'] / totalpeaks202['N'] 

# calculate percentage of N's that are deamidated
totalpeaks202['% Q w/ deam'] = totalpeaks202['q-deam'] / totalpeaks202['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks202['NAAF denom.'] = totalpeaks202['Total area'] / totalpeaks202['Total length']

# write modified dataframe to new txt file
totalpeaks202.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/202_BY_T00_GD_DN/202_BY_T00_GD_DN50_totals.csv")

totalpeaks202.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,130,30,30,51,104,84,237,0,136,321,...,17,15,1,1.879408e+09,2444,1.0,0.092896,0.151515,0.025641,768988.672042


In [8]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 768988.672042

# use NAAF >50% ALC to get NAAF factor
peaks202['NAAF factor'] = (peaks202['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksAAPTM_202 = peaks202[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_202['A-NAAF50'] = peaksAAPTM_202['A'] * peaks202['NAAF factor']
peaksAAPTM_202['C-NAAF50'] = peaksAAPTM_202['C'] * peaks202['NAAF factor']
peaksAAPTM_202['D-NAAF50'] = peaksAAPTM_202['D'] * peaks202['NAAF factor']
peaksAAPTM_202['E-NAAF50'] = peaksAAPTM_202['E'] * peaks202['NAAF factor']
peaksAAPTM_202['F-NAAF50'] = peaksAAPTM_202['F'] * peaks202['NAAF factor']
peaksAAPTM_202['G-NAAF50'] = peaksAAPTM_202['G'] * peaks202['NAAF factor']
peaksAAPTM_202['H-NAAF50'] = peaksAAPTM_202['H'] * peaks202['NAAF factor']
peaksAAPTM_202['I-NAAF50'] = peaksAAPTM_202['I'] * peaks202['NAAF factor']
peaksAAPTM_202['K-NAAF50'] = peaksAAPTM_202['K'] * peaks202['NAAF factor']
peaksAAPTM_202['L-NAAF50'] = peaksAAPTM_202['L'] * peaks202['NAAF factor']
peaksAAPTM_202['M-NAAF50'] = peaksAAPTM_202['M'] * peaks202['NAAF factor']
peaksAAPTM_202['N-NAAF50'] = peaksAAPTM_202['N'] * peaks202['NAAF factor']
peaksAAPTM_202['P-NAAF50'] = peaksAAPTM_202['P'] * peaks202['NAAF factor']
peaksAAPTM_202['Q-NAAF50'] = peaksAAPTM_202['Q'] * peaks202['NAAF factor']
peaksAAPTM_202['R-NAAF50'] = peaksAAPTM_202['R'] * peaks202['NAAF factor']
peaksAAPTM_202['S-NAAF50'] = peaksAAPTM_202['S'] * peaks202['NAAF factor']
peaksAAPTM_202['T-NAAF50'] = peaksAAPTM_202['T'] * peaks202['NAAF factor']
peaksAAPTM_202['V-NAAF50'] = peaksAAPTM_202['V'] * peaks202['NAAF factor']
peaksAAPTM_202['W-NAAF50'] = peaksAAPTM_202['W'] * peaks202['NAAF factor']
peaksAAPTM_202['Y-NAAF50'] = peaksAAPTM_202['Y'] * peaks202['NAAF factor']

# multiply the NAAF50 factor by the PTMN normalize its abundance by peak area and peptide length

peaksAAPTM_202['ccarb-NAAF50'] = peaksAAPTM_202['c-carb'] * peaksAAPTM_202['NAAF factor']
peaksAAPTM_202['moxid-NAAF50'] = peaksAAPTM_202['m-oxid'] * peaksAAPTM_202['NAAF factor']
peaksAAPTM_202['ndeam-NAAF50'] = peaksAAPTM_202['n-deam'] * peaksAAPTM_202['NAAF factor']
peaksAAPTM_202['qdeam-NAAF50'] = peaksAAPTM_202['q-deam'] * peaksAAPTM_202['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_202.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/202_BY_T00_GD_DN/202_BY_T00_GD_DN50_naaf.csv")

peaksAAPTM_202.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,VLEGNEQFLNAAK,0.037512,2,0,0,2,1,1,0,1,...,0.000000,0.000000,0.000000,0.037512,0.0,0.0,0.0,0.0,0.0,0.0
1,LSSPATLNSR,11.898745,1,0,0,0,0,0,0,0,...,11.898745,35.696235,11.898745,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,LSSPATLNSR,11.898745,1,0,0,0,0,0,0,0,...,11.898745,35.696235,11.898745,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,LSSPATLDSR,0.095840,1,0,1,0,0,0,0,0,...,0.095840,0.287520,0.095840,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,LSSPATLNSR,11.898745,1,0,0,0,0,0,0,0,...,11.898745,35.696235,11.898745,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMN

index = ['sample total']

data = {'NAAF': peaksAAPTM_202['NAAF factor'].sum(),
        'A': peaksAAPTM_202['A-NAAF50'].sum(),
        'C': peaksAAPTM_202['C-NAAF50'].sum(),
        'D': peaksAAPTM_202['D-NAAF50'].sum(),
        'E': peaksAAPTM_202['E-NAAF50'].sum(),
        'F': peaksAAPTM_202['F-NAAF50'].sum(),
        'G': peaksAAPTM_202['G-NAAF50'].sum(),
        'H': peaksAAPTM_202['H-NAAF50'].sum(),
        'I': peaksAAPTM_202['I-NAAF50'].sum(),
        'K': peaksAAPTM_202['K-NAAF50'].sum(),
        'L': peaksAAPTM_202['L-NAAF50'].sum(),
        'M': peaksAAPTM_202['M-NAAF50'].sum(),
        'N': peaksAAPTM_202['N-NAAF50'].sum(),
        'P': peaksAAPTM_202['P-NAAF50'].sum(),
        'Q': peaksAAPTM_202['Q-NAAF50'].sum(),
        'R': peaksAAPTM_202['R-NAAF50'].sum(),
        'S': peaksAAPTM_202['S-NAAF50'].sum(),
        'T': peaksAAPTM_202['T-NAAF50'].sum(),
        'V': peaksAAPTM_202['V-NAAF50'].sum(),
        'W': peaksAAPTM_202['W-NAAF50'].sum(),
        'Y': peaksAAPTM_202['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_202['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_202['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_202['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_202['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 768988.672042

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/202_BY_T00_GD_DN/202_BY_T00_GD_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,288.750993,199.085361,2.380971,1.33978,3.557737,8.700874,7.505703,13.857418,0.0,6.432317,...,6.469923,2.380971,0.886708,0.807831,0.001434,1.0,0.101689,0.014689,0.000271,0.000375


## Export stripped peptides >50% ALC

In [10]:
##### keep only stripped peptide column 
pep50 = peaks202[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/202_BY_T00_GD_DN/202_BY_T00_GD_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/202_BY_T00_GD_DN/202_BY_T00_GD_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/202_BY_T00_GD_DN/202_BY_T00_GD_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks202['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks202[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 244
# nonredundant stripped Peaks peptides >50% ALC 132
average peptide length Peaks peptides >50% ALC 10.01639344262295
# redundant Peaks peptides >50% ALC 244
# nonredundant Peaks peptides 133


,stripped_peptide
0,VLEGNEQFLNAAK
1,LSSPATLNSR
2,LSSPATLNSR
3,LSSPATLDSR
4,LSSPATLNSR
